# Applying Machine Learning to Stock Price Prediction
#### Donovan Wood, James Ohr, Andre Xiao  
## Abstract
* What problem did we address?
* What approach(es) did we use to address it?
* What are the big-picture results?
## Introduction
* Prompt: "Your introduction should describe the big-picture problem that you aimed to address in your project. What’s the problem to be solved, and why is it important? Who has tried solving this problem already, and what did they do? I would expect most introductions to reference no fewer than 2 scholarly studies that attempted similar tasks, although 5 is probably a better target."

In this blog post, we train machine learning models on historical stock market data to predict future stock price movements. This is a highly popular problem to address because of the potential for significant monetary gain. This is an important problem societally because stock markets are mechanisms of price discovery: they answer the question "What is a company worth?" Finding the right answer to that question allows society to correctly allocate more or less capital (money) to that company. On an individual level, this is an important problem to us as the authors because it's the problem for all quant trading: making a profitable model. 

An enormous body of literature within and without computer science exists for stock market prediction. Among the papers most relevant to our work are Bhandari et al. (2022) and Zhang (2022).

@gunduzEfficientPrediction2021 applies LSTM and ensemble learning (Light-GBM) models to predict the hourly directions of eight banking stocks in Borsa Istanbul. He achieved up to maximum success rate of 0.685 using individual features of bank stocks and LSTM.

@bhandariLSTM2022 apply single-layer and multi-layer LSTM models to the problem of predicting the S&P 500, the index of the largest 500 publicly traded companies in America. Their single-layer LTSM model with 150 neurons is their best performing specification. Their set of predicted values have an average correlation coefficient of 0.9976 with actual S&P index values.

@zhangPrediction2022 finds the LSTM network model does not perform better than other models when applied to a short forecasting horizon (1 to 10 days). Zhang's "other models" are linear regression, eXtreme gradient boosting (XGBoost), last value, and moving average.

We take some of the "best practices" we observe in the above papers, specifically benchmarking with last value and calculating accuracy with R, RMSE, and MAPE. Unlike the mentioned papers, we will be focusing on single stocks and attempting to build a model that outperforms the last value benchmark.


## Values

The potential users are anyone interested in making profitable trades in the stock market. They are the individuals most likely to directly benefit from our work. Nonusers who could be affected by our work are those engaged in the stock market. The obvious affected nonusers are those on the opposite side of each trade as a user. In every trade, there's a buyer and a seller, so in every trade, there's a winner and a loser. These opposing nonusers are the individuals who are most likely to be harmed by the success of our program. 

Ultimately, the point of the back and forth of markets is price discovery: to help society find the right prices of different companies. This leads to another nonuser effect: with better price discovery and more efficient markets, companies will raise money at prices that are closer to some "true" value, which is loosely defined as a value that best reflects the fundamental valuation of the company. Our model does not attempt to predict a true fundamental value for a company, but by making accurate predictions for the next day's price, it should accelerate the market's convergence to an appropriate value.

A useful financial trading model should lead to a net societal benefit because better financial markets mean more or less money going to companies and therefore projects, leading to something closer to an "optimal" allocation of money in society.

We are personally motivated to work on this project because of personal interest, professional relevance, and the difficulty of the problem. All three of us personally invest in the stock market. Two of us (Donovan & James) are double majors in economics and have had experience working in the financial services industry. Andre is interested in pursuing a master's in financial engineering after Middlebury. The problem itself is also inherently challenging: financial markets are constantly adapting and changing, making the findings of previous literature increasingly likely over time to be less applicable to today's markets. This forces us to adopt new techniques. 

### Materials and Methods 

'XOM', 'CVX', 'COP', 'EPD', 'PXD', 'EOG', 'DUK', 'MPC', 'SLB', 'PSX'

## Your Data 

Our data was sourced from Yahoo Finance. We used the `yfinance` library to download historical stock price data for our 10 different stocks. We chose to focus on US-based oil companies. These companies are Exxon Mobil (XOM), Chevron (CVX), ConocoPhillips (COP), Enterprise Products Partners (EPD), Pioneer Natural Resources (PXD), EOG Resources (EOG), Duke Energy (DUK), Marathon Petroleum (MPC), Schlumberger (SLB), and Phillips 66 (PSX). We downloaded the data from May 7th, 2014 to May 7th, 2024. 




## Your Approach

# What features of your data you used as predictors for your models, and what features (if any) you used as targets.

We used `SMA_20`, `SMA_50`, `Std_Dev`, `Z_Score`, `RSI`, `Close`, `TTM_P/E` as predictors for our models. 

The `SMA_20` and `SMA_50` are the 20-day and 50-day simple moving averages of the stock price. The `Std_Dev` is the standard deviation of the stock price. The `Z_Score` is the z-score of the stock price. The `RSI` is the relative strength index of the stock price. The `Close` is the closing price of the stock. The `TTM_P/E` is the trailing twelve months price-to-earnings ratio of the stock.

We used the `Close` price as the target variable for our model.

# Whether you subset your data in any way, and for what reasons.

We used a standard scaler for scaling our data in order to ensure that the data was normalized. We also used a train-test split of 90-10 in order to train our model on 90% of the data and test it on the remaining 10%. We used a 10-year window to predict stock price movements from May 7th, 2024. We used the closing price of the stock as the target variable for our model.

# What model(s) you used trained on your data, and how you chose them.

Originally, we used rather simplistic models like logistic regression, Random Forest, and SVM in order to predict stock price movements. We utilized Recursive Feature Elimination (RFE) in order to determine the optimal features for prediction for each model. However, we found that these models were not able to predict stock price movements consistently with much accuracy. We then decided to use a Long Short-Term Memory (LSTM) model to predict stock price movements. `LSTM` models are a type of recurrent neural network (RNN) with the addition of "gates" notably the `input`, `forget` and `output` gates. These gates allow for the model to determine what information to retain or discard at each timestep, mitigating the vanishing descent issue found in traditional recurrent neural networks. The LSTM model accounts for the shortfalls of an RNN by capturing long-term dependencies in the data.

The forget gate determines which information is either retained or discarded at each time step. It accepts the output from the previous time step $h_{t-1}$ and the input $x_t$ at the current time step. The forget gate is defined as:

$$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

The input gate determines which information is stored in the cell state. It avoids feeding the unimportant information into the current memory cell. It has three different components: 

1) Getting the state of the cell that must be updated.
2) Create a new cell state
3) Update the cell state to the current cell state

These are defined as: 

\begin{aligned}
i_{t} &= \sigma(W_{t} \cdot [h_{t-1}, x_{t}] + b_{i}) \\
\widetilde{C}_{t} &= \tanh(W_{c} \cdot [h_{t-1}, x_{t}] + b_{c}) \\
C_{t} &= f_{t} \ast C_{t-1} + i_{t} \ast \widetilde{C}_{t}
\end{aligned}

The output gate determines how much of the newly created cell state will be discarded and how much will be passed to the output. It is defined as:

$$o_{t} = \sigma(W_{o} \cdot [h_{t-1}, x_{t}] + b_{o})$$

This output information is firstly determined by a sigmoid layer, then the newly created cell state is processed by a tanh layer. The output is then multiplied by the sigmoid layer to determine the final output of the LSTM cell.

Which is defined as:

$$h_{t} = o_{t} \ast \tanh(C_{t})$$

Taking this all into account, the LSTM model is able to retain information from previous time steps and use it to predict future stock price movements while disregarding irrelevant information.

# How you trained your models, and on what hardware.

We trained our model using our own personal devices. We used the `Adam` optimizer with a learning rate of 0.001. We trained the model for 1000 epochs for each stock in our dataset (10 total). We used the `mean_squared_error` loss function to train the model.

# How you evaluated your models (loss, accuracy, etc), and the size of your test set.

We evaluated the accuracy of our model by comparing the predicted stock price movements to the actual stock price movements. We used the `mean_squared_error` loss function to evaluate the model. If the model predicted the next days price to be positive, we would purchase the stock at the closing price and sell it at the closing price the next day. If the model predicted the next days price to be negative, we would short the stock at the closing price and buy it back at the closing price the next day. We would then calculate the profit or loss percent change for each stock and compare it to the last value benchmark. 





## LSTM on 10 Biggest Energy Companies

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.autograd import Variable 
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
%load_ext autoreload
%autoreload 2
from lstm_model import LSTMModel
from lstm_data import prepare_data

In [ ]:
tickers = ['XOM', 'CVX', 'COP', 'EPD', 'PXD', 'EOG', 'DUK', 'MPC', 'SLB', 'PSX']
start = '2014-05-06'
end = '2024-05-07'
period = '10y'

In [ ]:
# preps data, see lstm_data.py, prints size of each ticker's dataset
X_train, y_train, X_test, y_test, X_scalers, y_scalers, batch_size = prepare_data(tickers, start_date=start, end_date=end, test_size=0.1)

In [ ]:
features = ['SMA_20', 'SMA_50', 'Std_Dev', 'Z_Score', 'RSI', 'Close', 'TTM_P/E']

X_train_tensors = Variable(torch.Tensor(np.array(X_train[features])))
y_train_tensors = Variable(torch.Tensor(y_train.values))
X_train_final = torch.reshape(X_train_tensors, (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))

In [ ]:
num_epochs = 1000 # 1000 epochs
learning_rate = 0.001 # 0.001 lr

input_size = X_train_final.shape[2] # number of features
hidden_size = 32 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers
batch_size = batch_size
window = 1 # number of windows, leave at 1, basically can ignore

num_classes = 1 # number of output classes

In [ ]:
# split data by ticker
data_loader_train = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_train_final, y_train_tensors),
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
lstm = LSTMModel(num_classes, input_size, hidden_size, num_layers, seq_length=window, batch_size=batch_size) #our lstm class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) # ADAM optimizer

**Below takes about 7 min to run.**

In [ ]:
for epoch in range(num_epochs):
  for i, data in enumerate(data_loader_train):
    X_, y_ = data
    outputs = lstm.forward(X_) #forward pass
    optimizer.zero_grad() #calculate the gradient, manually setting to 0
  
    # obtain the loss function
    loss = criterion(outputs, y_.reshape(y_.size(0)*y_.size(1), 1))
  
    loss.backward() #calculates the loss of the loss function
  
    optimizer.step() #improve from loss, i.e backprop
    # if (i + 1) % 50 == 0:
    #     print(f"Epoch {epoch}, batch {i:>3}, loss on batch: {loss.item():.3f}")
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def evaluate_lstm(model, X_test, y_test, X_scaler, y_scaler, features):
    ticker = X_test['Ticker'].iloc[0] # get ticker
    X_test_tensors = Variable(torch.Tensor(np.array(X_test[features]))) # prepare for lstm
    X_test_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], 1, X_test_tensors.shape[1])) # prepare for lstm

    test_predict = model(X_test_final).data.numpy() # predict
    test_predict = y_scaler.inverse_transform(test_predict) # reverse transform back to original scale
    cols = X_test.columns[X_test.columns != 'Ticker']
    X_test = pd.DataFrame(X_scaler.inverse_transform(X_test[cols]), index=X_test.index, columns=cols) # reverse transform X_test back to og scale
    predicted_price = pd.DataFrame(test_predict)
    predicted_price.columns = ['Predicted_Price']
    predicted_price.size
    idx = X_test.index[:predicted_price.size]
    predicted_price.index = idx # fix index of predicted prices
    X_test = pd.concat([X_test, predicted_price], ignore_index=False, axis=1) 
    X_test = X_test.dropna()
    X_test['Actual_Signal'] = (X_test['Returns'].shift(-1) > 0).astype(int) # actual buy/sell signal based on daily returns
    X_test['Predicted_Returns'] = X_test['Predicted_Price'].pct_change()
    X_test['Predicted_Signal'] = (X_test['Predicted_Returns'] > 0).astype(int) # predicted buy/sell signal based on predicted returns
    X_test['Strategy_Returns'] = X_test['Returns'] * X_test['Predicted_Signal'].shift(1) # calculate daily strategy returns
    cumulative_strategy_returns = (X_test['Strategy_Returns'].fillna(0) + 1).cumprod()
    returns = X_test.loc[X_test.index, 'Returns']
    returns.iloc[0] = 0
    cumulative_stock_returns = (returns + 1).cumprod()
    accuracy = (X_test['Actual_Signal'] == X_test['Predicted_Signal']).mean()
    print(f'{ticker} Accuracy: {accuracy}')

    # plot stock price
    plt.figure(figsize=(10,5))
    plt.plot(X_test['Predicted_Price'].shift(1), label='Predicted Price')
    plt.plot(X_test['Close'], label='Actual Price')
    plt.title(f'{ticker} Price')
    plt.legend();

    # plot returns
    plt.figure(figsize=(10,5))
    plt.plot(cumulative_strategy_returns, label='Strategy Returns')
    plt.plot(cumulative_stock_returns, label='Stock Returns')
    plt.title(f'{ticker} Returns')
    plt.legend();

    # plot confusion matrix
    cm = confusion_matrix(X_test['Actual_Signal'], X_test['Predicted_Signal'])
    cm_display = ConfusionMatrixDisplay(cm, display_labels=['Sell', 'Buy'])
    cm_display.plot();
    plt.title(f'{ticker} Confusion Matrix')

    return cumulative_strategy_returns, cumulative_stock_returns

In [ ]:
cum_strat_returns_list = np.array([])
cum_stock_returns_list = np.array([])
for i in range(10):
    cum_strat_returns, cum_stock_returns = evaluate_lstm(lstm, X_test[i], y_test[i], X_scalers[i], y_scalers[i], features)
    cum_strat_returns_list = np.append(cum_strat_returns_list, cum_strat_returns.iloc[-1])
    cum_stock_returns_list = np.append(cum_stock_returns_list, cum_stock_returns.iloc[-1])

In [ ]:
total_strat_returns = (cum_strat_returns_list).mean()
total_stock_returns = (cum_stock_returns_list).mean()

print(f'Portfolio Returns: {total_strat_returns}')
print(f'Stock Returns: {total_stock_returns}')

In [ ]:
from torchinfo import summary

summary(lstm)

## Live Mock Testing

In [ ]:
from sklearn.preprocessing import StandardScaler

start = '2014-05-06'
end = '2024-05-08'


X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []
X_scalers = []
y_scalers = []
for t in tickers:
    ticker_data = yf.Ticker(t)
    #data = ticker_data.history(period=period)
    data = yf.download(t, start=start, end=end)

    # Calculate moving averages and std
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['Std_Dev'] = data['Close'].rolling(window=20).std()

    # Calculate the z-score
    data['Z_Score'] = (data['Close'] - data['SMA_20']) / data['Std_Dev']

    # Calculate RSI
    delta = data['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.ewm(com=13, adjust=False).mean()
    ema_down = down.ewm(com=13, adjust=False).mean()
    rs = ema_up / ema_down

    data['RSI'] = 100 - (100 / (1 + rs))
    # Calculate TTM EPS and P/E
    eps = ticker_data.get_earnings_dates(limit=60)
    eps.index = eps.index.date
    eps = eps.loc[~eps.index.duplicated(keep='first'), :]
    data.index = data.index.date
    eps = eps[(eps.index >= (data.index[0]-relativedelta(years=1))) & (eps.index <= data.index[-1])]
    if t == 'DUK':
        eps.loc[eps.index == pd.to_datetime('2024-05-07').date(), 'Reported EPS'] = 1.44
    eps = eps.iloc[::-1]
    eps['TTM'] = eps['Reported EPS'].rolling(window=4).sum()
    
    idx = pd.date_range(eps.index[0], eps.index[-1])
    eps = eps.reindex(idx.date, fill_value=np.nan)
    data['TTM_EPS'] = eps['TTM'].copy()
    data[data['TTM_EPS'].notna()]
    data['TTM_EPS'] = data['TTM_EPS'].ffill()
    data['TTM_EPS'] = data['TTM_EPS'].fillna(eps['TTM'].loc[eps['TTM'].notna()].iloc[0])
    data['TTM_P/E'] = data['Close'] / data['TTM_EPS']

    # Calculate the daily returns
    data['Returns'] = data['Close'].pct_change()

    # Drop any NaNs

    # If stock price goes up or down
    data['Target'] = data['Close'].shift(-1)

    data = data.dropna()
    #features = ['Ticker', 'SMA_20', 'SMA_50', 'Std_Dev', 'Z_Score', 'RSI', 'Returns']
    X = data.loc[:, data.columns != 'Target']
    y = data.iloc[:, (data.shape[1]-1):(data.shape[1])]
    #X = X.dropna()
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=False)
    
    ss1 = StandardScaler()
    ss2 = StandardScaler()

    X_train_ss = pd.DataFrame(ss1.fit_transform(X), index=X.index, columns=X.columns) # fit ss and transform X_train
    y_train_ss = pd.DataFrame(ss2.fit_transform(y), index=y.index, columns=y.columns) # fit mm and transform y_train

    X_train_ss['Ticker'] = t
    print(X_train_ss.shape)
    X_train_list.append(X_train_ss)
    y_train_list.append(y_train_ss)
    X_scalers.append(ss1)
    y_scalers.append(ss2)
batch_size = X_train_list[0].shape[0]

In [ ]:
X_train = pd.concat(X_train_list)
y_train = pd.concat(y_train_list)

features = ['SMA_20', 'SMA_50', 'Std_Dev', 'Z_Score', 'RSI', 'Close', 'TTM_P/E']

X_train_tensors = Variable(torch.Tensor(np.array(X_train[features])))
y_train_tensors = Variable(torch.Tensor(y_train.values))
X_train_final = torch.reshape(X_train_tensors, (X_train_tensors.shape[0], 1, X_train_tensors.shape[1]))


In [ ]:
num_epochs = 1000 # 1000 epochs
learning_rate = 0.001 # 0.001 lr

input_size = X_train_final.shape[2] # number of features
hidden_size = 32 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers
batch_size = batch_size
window = 1 # number of windows, leave at 1, basically can ignore

num_classes = 1 # number of output classes

In [ ]:
# split data by ticker
data_loader_train = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(X_train_final, y_train_tensors),
    batch_size=batch_size,
    shuffle=True
)

In [ ]:
lstm_live = LSTMModel(num_classes, input_size, hidden_size, num_layers, seq_length=window, batch_size=batch_size) #our lstm class 
criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm_live.parameters(), lr=learning_rate) # ADAM optimizer

In [ ]:
for epoch in range(num_epochs):
  for i, data in enumerate(data_loader_train):
    X_, y_ = data
    outputs = lstm_live.forward(X_) #forward pass
    optimizer.zero_grad() #calculate the gradient, manually setting to 0
  
    # obtain the loss function
    loss = criterion(outputs, y_.reshape(y_.size(0)*y_.size(1), 1))
  
    loss.backward() #calculates the loss of the loss function
  
    optimizer.step() #improve from loss, i.e backprop
    # if (i + 1) % 50 == 0:
    #     print(f"Epoch {epoch}, batch {i:>3}, loss on batch: {loss.item():.3f}")
  if epoch % 100 == 0:
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
from joblib import dump, load

In [ ]:
dump(lstm_live, 'lstm.joblib')

In [ ]:
from sklearn.preprocessing import StandardScaler
df_pred_list = []
for j in range(7, 10):
    tickers = ['XOM', 'CVX', 'COP', 'NEE', 'SO', 'EOG', 'DUK', 'MPC', 'SLB', 'PSX']
    start = '2014-05-06'
    end = f'2024-05-0{j}'

    X_train_list = []
    y_train_list = []
    X_test_list = []
    y_test_list = []
    X_scalers = []
    y_scalers = []
    for t in tickers:
        ticker_data = yf.Ticker(t);
        #data = ticker_data.history(period=period)
        data = yf.download(t, start=start, end=end);

        # Calculate moving averages and std
        data['SMA_20'] = data['Close'].rolling(window=20).mean()
        data['SMA_50'] = data['Close'].rolling(window=50).mean()
        data['Std_Dev'] = data['Close'].rolling(window=20).std()

        # Calculate the z-score
        data['Z_Score'] = (data['Close'] - data['SMA_20']) / data['Std_Dev']

        # Calculate RSI
        delta = data['Close'].diff()
        up = delta.clip(lower=0)
        down = -1 * delta.clip(upper=0)
        ema_up = up.ewm(com=13, adjust=False).mean()
        ema_down = down.ewm(com=13, adjust=False).mean()
        rs = ema_up / ema_down

        data['RSI'] = 100 - (100 / (1 + rs))
        # Calculate TTM EPS and P/E
        eps = ticker_data.get_earnings_dates(limit=60)
        eps.index = eps.index.date
        eps = eps.loc[~eps.index.duplicated(keep='first'), :]
        data.index = data.index.date
        eps = eps[(eps.index >= (data.index[0]-relativedelta(years=1))) & (eps.index <= data.index[-1])]
        if t == 'DUK':
            eps.loc[eps.index == pd.to_datetime('2024-05-07').date(), 'Reported EPS'] = 1.44
        eps = eps.iloc[::-1]
        eps['TTM'] = eps['Reported EPS'].rolling(window=4).sum()
        
        
        idx = pd.date_range(eps.index[0], eps.index[-1])
        eps = eps.reindex(idx.date, fill_value=np.nan)
        data['TTM_EPS'] = eps['TTM'].copy()
        data[data['TTM_EPS'].notna()]
        data['TTM_EPS'] = data['TTM_EPS'].ffill()
        data['TTM_EPS'] = data['TTM_EPS'].fillna(eps['TTM'].loc[eps['TTM'].notna()].iloc[0])
        data['TTM_P/E'] = data['Close'] / data['TTM_EPS']

        #X_test_list.append(data.iloc[-1])

        # Calculate the daily returns
        data['Returns'] = data['Close'].pct_change()

        # Drop any NaNs

        # If stock price goes up or down
        data['Target'] = data['Close'].shift(-1)

        #data = data.dropna()
        #features = ['Ticker', 'SMA_20', 'SMA_50', 'Std_Dev', 'Z_Score', 'RSI', 'Returns']
        X = data.loc[:, data.columns != 'Target']
        y = data.iloc[:, (data.shape[1]-1):(data.shape[1])]
        X = X.dropna()
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=False)
        
        ss1 = StandardScaler()
        ss2 = StandardScaler()

        X_test_ss = pd.DataFrame(ss1.fit_transform(X), index=X.index, columns=X.columns) # fit ss and transform X_train
        y_test_ss = pd.DataFrame(ss2.fit_transform(y), index=y.index, columns=y.columns) # fit mm and transform y_train

        X_test_ss['Ticker'] = t
        print(X_train_ss.shape)
        X_test_list.append(X_test_ss)
        y_test_list.append(y_test_ss)
        X_scalers.append(ss1)
        y_scalers.append(ss2)
    batch_size = X_test_list[0].shape[0]

    features = ['SMA_20', 'SMA_50', 'Std_Dev', 'Z_Score', 'RSI', 'Close', 'TTM_P/E']
    next_day_predict_list = []

    for i, test in enumerate(X_test_list):
        X_test_tensors = Variable(torch.Tensor(np.array(test[features])))
        X_test_final = torch.reshape(X_test_tensors, (X_test_tensors.shape[0], 1, X_test_tensors.shape[1]))
        predict = lstm_live(X_test_final).data.numpy()
        predict = y_scalers[i].inverse_transform(predict)
        predicted_price = pd.DataFrame(predict)

        cols = test.columns[test.columns != 'Ticker']
        test = pd.DataFrame(X_scalers[i].inverse_transform(test[cols]), index=test.index, columns=cols)

        predicted_price.columns = ['Predicted_Price']
        predicted_price.size
        idx = test.index[:predicted_price.size]
        predicted_price.index = idx

        test = pd.concat([test, predicted_price], ignore_index=False, axis=1)
        test['Predicted_Returns'] = test['Predicted_Price'].pct_change()
        test['Predicted_Signal'] = (test['Predicted_Returns'] > 0)*1
        test['Actual_Signal'] = (test['Returns'].shift(-1) > 0)*1
        test['Ticker'] = tickers[i]

        X_test_list[i] = test
        test = test.loc[test.index == pd.to_datetime(f'2024-05-0{j-1}').date(), :]
        next_day_predict_list.append(test[['Ticker', 'Close', 'Predicted_Price', 'Predicted_Returns', 'Predicted_Signal', 'Actual_Signal']])
        
    df_pred_list.append(pd.concat(next_day_predict_list))



In [ ]:
def add_current(df_pred):    
    current = np.array([])
    for t in tickers:
        data = yf.download(t, period='1d')
        #data = data.loc[data.index == pd.to_datetime('2024-05-07'), :]
        current = np.append(current, data['Close'])

    df_pred['Actual_Price'] = current
    df_pred['Actual_Signal'] = (df_pred['Actual_Price'] > df_pred['Close'])*1
    return df_pred

In [ ]:
may7_pred = add_current(df_pred_list[0])
may7_pred

In [ ]:
(may7_pred['Predicted_Signal'] == may7_pred['Actual_Signal']).mean()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(may7_pred['Actual_Signal'], may7_pred['Predicted_Signal'])
cm_display = ConfusionMatrixDisplay(cm, display_labels=['Sell', 'Buy'])
cm_display.plot();

In [ ]:
may8_pred = add_current(df_pred_list[1])
may8_pred

In [ ]:
(may8_pred['Predicted_Signal'] == may8_pred['Actual_Signal']).mean()

In [ ]:
cm = confusion_matrix(may8_pred['Actual_Signal'], may8_pred['Predicted_Signal'])
cm_display = ConfusionMatrixDisplay(cm, display_labels=['Sell', 'Buy'])
cm_display.plot();